### Read training, dev and unlabeled test data

The following provides a starting code (Python 3) of how to read the labeled training and dev sentence pairs, and unlabeled test sentence pairs, into lists.

In [1]:
import csv

In [2]:
train, dev, test = [], [], []

In [3]:
with open('./data/pnli_train.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[2] will be the label (0 or 1). x[0] and x[1] will be the sentence pairs.
        train.append(x)
print (len(train))
print (train[:3])

5983
[['Sometimes do exercise.', 'A person typically desire healthy life.', '1'], ['Who eats junk foods.', 'A person typically desire healthy life.', '0'], ['A person is sick.', 'A person typically desire healthy life.', '1']]


In [4]:
with open('./data/pnli_dev.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[2] will be the label (0 or 1). x[0] and x[1] will be the sentence pairs.
        dev.append(x)
print (len(dev))
print (dev[:3])

1055
[['A person is looking for accuracy.', 'A person typically desires accurate results.', '1'], ['A person does not care for accuracy.', 'A person typically desires accurate results.', '0'], ['The person double checks their data.', 'A person typically desires accurate results.', '1']]


In [5]:
with open('./data/pnli_test_unlabeled.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[0] and x[1] will be the sentence pairs.
        test.append(x)
print (len(test))
print (test[:3])

4850
[['The people want to have a romantic and pleasant feel.', 'People typically does desire to smell violets.'], ['The contract is to buy products from you.', 'Getting contract typically cause to make money or spend money.'], ['Train station is closed.', 'Line can typically be used to move train along tracks.']]


### Main Code Body

You may choose to experiment with different methods using your program. However, you need to embed the training and inference processes at here. We will use your prediction on the unlabeled test data to grade, while checking this part to understand how your method has produced the predictions.

In [6]:
# Eventually, results need to be a list of 2028 0 or 1's
results = []

In [7]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 8.0 MB/s 
     |████████████████████████████████| 895 kB 63.6 MB/s 
     |████████████████████████████████| 596 kB 65.3 MB/s 
     |████████████████████████████████| 77 kB 9.3 MB/s 
     |████████████████████████████████| 6.6 MB 54.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
import random
import torch
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)

tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli") 

model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")
model.to(device)

learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)

Downloading:   0%|          | 0.00/688 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [10]:
def encode_data(tokenizer, questions, answers, max_length):
    input_ids = []
    attention_masks = []

    for question, answer in zip(questions, answers):
        encoded_data = tokenizer.encode_plus(question, 
                                             answer, 
                                             max_length=max_length, 
                                             pad_to_max_length=True, 
                                             truncation_strategy="longest_first")
        input_id = encoded_data["input_ids"]
        attention_mask = encoded_data["attention_mask"]

        input_ids.append(input_id)
        attention_masks.append(attention_mask)

    return np.array(input_ids), np.array(attention_masks)

train_question = []
train_answer = []
train_label = []

for i in range(len(train)):
  train_question.append(train[i][0])
  train_answer.append(train[i][1])
  train_label.append(train[i][2])

dev_question = []
dev_answer = []
dev_label = []

for i in range(len(dev)):
  dev_question.append(dev[i][0])
  dev_answer.append(dev[i][1])
  dev_label.append(dev[i][2])

train_label = [int(i) for i in train_label]
dev_label = [int(i) for i in dev_label]

# Encoding data
max_seq_length = 64 #change
input_ids_train, attention_masks_train = encode_data(tokenizer, train_question, train_answer, max_seq_length)
input_ids_dev, attention_masks_dev = encode_data(tokenizer, dev_question, dev_answer, max_seq_length)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [11]:
batch_size = 32 #change

train_features = (input_ids_train, attention_masks_train, train_label)
dev_features = (input_ids_dev, attention_masks_dev, dev_label)

train_features_tensors = [torch.tensor(feature, dtype = torch.long) for feature in train_features]
dev_features_tensors = [torch.tensor(feature, dtype = torch.long) for feature in dev_features]

train_dataset = TensorDataset(*train_features_tensors)
dev_dataset = TensorDataset(*dev_features_tensors)

train_sampler = RandomSampler(train_dataset)
dev_sampler = SequentialSampler(dev_dataset)

train_dataloader = DataLoader(train_dataset, sampler = train_sampler, batch_size = batch_size)
dev_dataloader = DataLoader(dev_dataset, sampler = dev_sampler, batch_size = batch_size)

In [12]:
epochs = 4
grad_acc_steps = 4

for epoch_i in range(0, epochs):

  # Training

  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')
  epoch_train_loss = 0
  model.train()
  model.zero_grad()

  for step, batch in enumerate(train_dataloader):

      input_ids = batch[0].to(device)
      attention_masks = batch[1].to(device)
      labels = batch[2].to(device)     

      outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)

      loss = outputs[0]
      loss = loss / grad_acc_steps
      epoch_train_loss += loss.item()

      loss.backward()
      
      if (step+1) % grad_acc_steps == 0: 
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) 
        optimizer.step()
        model.zero_grad()

  epoch_train_loss = epoch_train_loss / len(train_dataloader)
  print("")
  print("Average training loss: {0:.2f}".format(epoch_train_loss))          
  
  # Evaluation
  epoch_dev_accuracy = 0 
  print("")
  print("Running Validation...")
  model.eval()

  for batch in dev_dataloader:
    
    input_ids = batch[0].to(device)
    attention_masks = batch[1].to(device)
    labels = batch[2]
                
    with torch.no_grad():        
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
                    
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    
    predictions = np.argmax(logits, axis=1).flatten()
    labels = labels.numpy().flatten()
    
    epoch_dev_accuracy += np.sum(predictions == labels) / len(labels)

  epoch_dev_accuracy = epoch_dev_accuracy / len(dev_dataloader)
  print("")
  print("Average Validation Accuracy: {0:.4f}".format(epoch_dev_accuracy))


======== Epoch 1 / 4 ========
Training...

Average training loss: 0.10

Running Validation...

Average Validation Accuracy: 0.8939

======== Epoch 2 / 4 ========
Training...

Average training loss: 0.05

Running Validation...

Average Validation Accuracy: 0.9119

======== Epoch 3 / 4 ========
Training...

Average training loss: 0.04

Running Validation...

Average Validation Accuracy: 0.9147

======== Epoch 4 / 4 ========
Training...

Average training loss: 0.02

Running Validation...

Average Validation Accuracy: 0.9147


In [ ]:
def predict(question, answer):
  sequence = tokenizer.encode_plus(question, answer, return_tensors="pt")['input_ids'].to(device)
  
  logits = model(sequence)[0]
  probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
  proba_yes = round(probabilities[1], 2)
  proba_no = round(probabilities[0], 2)

  if proba_yes >= proba_no:
    return 1
  else:
    return 0

In [ ]:
for i in range(len(test)):
  ans = predict(test[i][0],test[i][1])
  results.append(ans)

### Output Prediction Result File

You will need to submit a prediction result file. It should have 2028 lines, every line should be either 0 or 1, which is your model's prediction on the respective test set instance.

In [ ]:
# suppose you had your model's predictions on the 2028 test cases read from test_enc_unlabeled.tsv, and 
#those results are in the list called 'results'
assert (len(results) == 4850)

In [ ]:
# make sure the results are not float numbers, but intergers 0 and 1
results = [int(x) for x in results]

In [ ]:
# write your prediction results to 'upload_predictions.txt' and upload that later
with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
    for x in results:
        fp.write(str(x) + '\n')